<a href="https://colab.research.google.com/github/MLDreamer/AIMathematicallyexplained/blob/main/Laplacian_Positional_Encoding_animations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy import sparse
from scipy.sparse.linalg import eigsh
import networkx as nx
import warnings
warnings.filterwarnings('ignore')


plt.style.use('dark_background')
CYAN = '#58C4DD'
MAGENTA = '#FF00FF'
YELLOW = '#FFFF00'
RED = '#FC6255'
GREEN = '#83C167'
BLUE = '#58C4DD'
ORANGE = '#FC9F5B'
PURPLE = '#CE6FFF'
WHITE = 'white'

print("=" * 80)
print("🎵 LAPLACIAN POSITIONAL ENCODING: ANIMATION GENERATOR")
print("=" * 80)
print("\nGenerating 8 pedagogical animations...")
print("Estimated time: ~30-35 minutes\n")

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def compute_laplacian_pe(adjacency, k=10):
    """Compute Laplacian Positional Encoding."""
    degrees = adjacency.sum(axis=1).A1 if sparse.issparse(adjacency) else adjacency.sum(axis=1)
    D = sparse.diags(degrees) if sparse.issparse(adjacency) else np.diag(degrees)
    L = D - adjacency

    if sparse.issparse(L):
        eigenvalues, eigenvectors = eigsh(L.tocsr(), k=k+1, which='SM')
    else:
        eigenvalues, eigenvectors = np.linalg.eigh(L)
        idx = eigenvalues.argsort()
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        eigenvalues = eigenvalues[:k+1]
        eigenvectors = eigenvectors[:, :k+1]

    return eigenvalues[1:], eigenvectors[:, 1:]

def create_circular_drum_mesh(n_r=30, n_theta=60):
    """Create circular mesh for drum visualization."""
    r = np.linspace(0, 1, n_r)
    theta = np.linspace(0, 2*np.pi, n_theta)
    R, Theta = np.meshgrid(r, theta)
    X = R * np.cos(Theta)
    Y = R * np.sin(Theta)
    return X, Y, R, Theta

# ============================================================================
# ANIMATION 1: TRANSFORMER_BLINDNESS
# ============================================================================

print("🎬 Animation 1/8: TRANSFORMER_BLINDNESS.gif")
print("   Concept: Transformer seeing graph as flat sequence")
print("   Duration: 12 seconds")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), facecolor='#0E1117')
for ax in [ax1, ax2]:
    ax.set_facecolor('#0E1117')

# Create a simple molecular graph (H2O structure)
def animate_transformer_blindness(frame):
    ax1.clear()
    ax2.clear()
    for ax in [ax1, ax2]:
        ax.set_facecolor('#0E1117')
        ax.axis('off')

    progress = frame / 60

    # Left: Actual H2O geometry (bent)
    H1_pos = np.array([-0.8, -0.6])
    O_pos = np.array([0, 0])
    H2_pos = np.array([0.8, -0.6])

    fade = min(progress * 2, 1.0)

    # Draw bonds
    ax1.plot([H1_pos[0], O_pos[0]], [H1_pos[1], O_pos[1]],
            'w-', linewidth=3, alpha=fade*0.5)
    ax1.plot([H2_pos[0], O_pos[0]], [H2_pos[1], O_pos[1]],
            'w-', linewidth=3, alpha=fade*0.5)

    # Draw atoms
    ax1.scatter(*H1_pos, s=500, c=CYAN, edgecolors=WHITE, linewidth=3,
               alpha=fade, zorder=10)
    ax1.scatter(*O_pos, s=800, c=RED, edgecolors=WHITE, linewidth=3,
               alpha=fade, zorder=10)
    ax1.scatter(*H2_pos, s=500, c=CYAN, edgecolors=WHITE, linewidth=3,
               alpha=fade, zorder=10)

    ax1.text(H1_pos[0], H1_pos[1]-0.3, 'H', ha='center', va='top',
            fontsize=20, color=WHITE, weight='bold', alpha=fade)
    ax1.text(O_pos[0], O_pos[1]+0.3, 'O', ha='center', va='bottom',
            fontsize=20, color=WHITE, weight='bold', alpha=fade)
    ax1.text(H2_pos[0], H2_pos[1]-0.3, 'H', ha='center', va='top',
            fontsize=20, color=WHITE, weight='bold', alpha=fade)

    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_title('ACTUAL GEOMETRY\nBent, 104.5° angle',
                 fontsize=14, color=GREEN, weight='bold', pad=20)

    # Right: How Transformer sees it (linear sequence)
    if progress > 0.3:
        seq_fade = (progress - 0.3) / 0.7

        positions = [(-1, 0), (0, 0), (1, 0)]
        colors = [CYAN, RED, CYAN]
        labels = ['H', 'O', 'H']

        for i, (pos, color, label) in enumerate(zip(positions, colors, labels)):
            ax2.scatter(*pos, s=600, c=color, edgecolors=WHITE, linewidth=3,
                       alpha=seq_fade, zorder=10)
            ax2.text(pos[0], pos[1]-0.3, label, ha='center', va='top',
                    fontsize=20, color=WHITE, weight='bold', alpha=seq_fade)

            # Position numbers
            ax2.text(pos[0], pos[1]+0.4, f'Pos {i+1}', ha='center', va='bottom',
                    fontsize=12, color=YELLOW, alpha=seq_fade)

        # Arrows showing sequence
        if seq_fade > 0.5:
            ax2.annotate('', xy=(0, 0), xytext=(-1, 0),
                        arrowprops=dict(arrowstyle='->', color=YELLOW,
                                      lw=2, alpha=seq_fade))
            ax2.annotate('', xy=(1, 0), xytext=(0, 0),
                        arrowprops=dict(arrowstyle='->', color=YELLOW,
                                      lw=2, alpha=seq_fade))

    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    ax2.set_title('WHAT TRANSFORMER SEES\nFlat sequence: 1 → 2 → 3',
                 fontsize=14, color=RED, weight='bold', pad=20)

    if progress > 0.7:
        fig.text(0.5, 0.02, 'The 104.5° angle is lost. Structure becomes invisible.',
                ha='center', fontsize=12, color=RED, weight='bold',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.9))

anim1 = FuncAnimation(fig, animate_transformer_blindness, frames=60, interval=200)
print("   Rendering...")
anim1.save('/content/TRANSFORMER_BLINDNESS.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ Saved\n")

# ============================================================================
# ANIMATION 2: KAC_DRUM
# ============================================================================

print("🎬 Animation 2/8: KAC_DRUM.gif")
print("   Concept: Circular drum vibrating (fundamental mode)")
print("   Duration: 10 seconds")

fig = plt.figure(figsize=(10, 10), facecolor='#0E1117')
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('#0E1117')

X, Y, R, Theta = create_circular_drum_mesh(30, 60)

def animate_kac_drum(frame):
    ax.clear()
    ax.set_facecolor('#0E1117')

    t = frame / 50 * 2 * np.pi

    # Fundamental mode: J_0(r) * cos(ωt)
    # Using approximation for Bessel function
    omega = 2.4048  # First zero of J_0
    Z = np.cos(omega * R) * np.cos(t) * 0.3

    # Color by height
    colors = cm.coolwarm((Z - Z.min()) / (Z.max() - Z.min()))

    surf = ax.plot_surface(X, Y, Z, facecolors=colors, alpha=0.9,
                          edgecolor='white', linewidth=0.3, shade=True)

    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-0.5, 0.5)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_zlabel('')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.view_init(elev=20, azim=frame*2)

    ax.set_title('KAC\'S DRUM: FUNDAMENTAL MODE\n"Can you hear the shape?"',
                fontsize=14, color=YELLOW, weight='bold', pad=20)

    # Add frequency label
    ax.text2D(0.5, 0.02, f'Frequency: ω₁ = {omega:.2f}\nTime: {t/(2*np.pi):.2f}T',
             transform=ax.transAxes, ha='center', fontsize=11, color=WHITE,
             bbox=dict(boxstyle='round', facecolor='black', alpha=0.9))

anim2 = FuncAnimation(fig, animate_kac_drum, frames=50, interval=200)
print("   Rendering...")
anim2.save('/content/KAC_DRUM.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ Saved\n")

# ============================================================================
# ANIMATION 3: GRAPH_VIBRATION
# ============================================================================

print("🎬 Animation 3/8: GRAPH_VIBRATION.gif")
print("   Concept: Graph 'plucked' showing vibration")
print("   Duration: 14 seconds")

# Create a cycle graph
n_nodes = 20
G = nx.cycle_graph(n_nodes)
pos = nx.circular_layout(G)
A = nx.adjacency_matrix(G).toarray()

# Compute eigenvectors
eigenvalues, eigenvectors = compute_laplacian_pe(A, k=10) # Changed k from 3 to 10

fig, ax = plt.subplots(figsize=(10, 10), facecolor='#0E1117')
ax.set_facecolor('#0E1117')

def animate_graph_vibration(frame):
    ax.clear()
    ax.set_facecolor('#0E1117')
    ax.axis('off')

    progress = frame / 70

    # Which eigenmode to show
    if progress < 0.33:
        mode = 0
        title = "FUNDAMENTAL MODE (φ₁)\nWhole graph vibrates together"
        t = progress / 0.33 * 2 * np.pi
    elif progress < 0.66:
        mode = 1
        title = "SECOND MODE (φ₂)\nGraph splits in half"
        t = (progress - 0.33) / 0.33 * 2 * np.pi
    else:
        mode = 2
        title = "THIRD MODE (φ₃)\nThree vibrating regions"
        t = (progress - 0.66) / 0.34 * 2 * np.pi

    # Displacement based on eigenmode
    eigenvector = eigenvectors[:, mode]
    displacements = eigenvector * np.cos(t) * 0.15

    # Adjust positions
    pos_vibrating = {}
    for i, (node, (x, y)) in enumerate(pos.items()):
        # Radial displacement
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        r_new = r + displacements[i]
        pos_vibrating[node] = (r_new * np.cos(theta), r_new * np.sin(theta))

    # Draw edges
    for edge in G.edges():
        x_coords = [pos_vibrating[edge[0]][0], pos_vibrating[edge[1]][0]]
        y_coords = [pos_vibrating[edge[0]][1], pos_vibrating[edge[1]][1]]
        ax.plot(x_coords, y_coords, 'w-', linewidth=2, alpha=0.3, zorder=1)

    # Draw nodes with colors based on eigenvector values
    node_colors = cm.coolwarm((eigenvector - eigenvector.min()) /
                              (eigenvector.max() - eigenvector.min()))

    for i, (node, (x, y)) in enumerate(pos_vibrating.items()):
        ax.scatter(x, y, s=300, c=[node_colors[i]],
                  edgecolors=WHITE, linewidth=2, zorder=10)

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_title(title, fontsize=14, color=YELLOW, weight='bold', pad=20)

    # Show eigenvalue
    ax.text(0.5, 0.02, f'Eigenvalue λ_{mode+1} = {eigenvalues[mode]:.3f}',
           transform=ax.transAxes, ha='center', fontsize=11, color=WHITE,
           bbox=dict(boxstyle='round', facecolor='black', alpha=0.9))

anim3 = FuncAnimation(fig, animate_graph_vibration, frames=70, interval=200)
print("   Rendering...")
anim3.save('/content/GRAPH_VIBRATION.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ Saved\n")

# ============================================================================
# ANIMATION 4: EIGENMODE_VISUALIZATION
# ============================================================================

print("🎬 Animation 4/8: EIGENMODE_VISUALIZATION.gif")
print("   Concept: Side-by-side comparison of first 3 eigenmodes")
print("   Duration: 10 seconds")

fig, axes = plt.subplots(1, 3, figsize=(18, 6), facecolor='#0E1117')
for ax in axes:
    ax.set_facecolor('#0E1117')
    ax.axis('off')

def animate_eigenmodes(frame):
    t = frame / 50 * 2 * np.pi

    for mode_idx, ax in enumerate(axes):
        ax.clear()
        ax.set_facecolor('#0E1117')
        ax.axis('off')

        eigenvector = eigenvectors[:, mode_idx]
        displacements = eigenvector * np.cos(t) * 0.15

        # Adjust positions
        pos_vibrating = {}
        for i, (node, (x, y)) in enumerate(pos.items()):
            r = np.sqrt(x**2 + y**2)
            theta = np.arctan2(y, x)
            r_new = r + displacements[i]
            pos_vibrating[node] = (r_new * np.cos(theta), r_new * np.sin(theta))

        # Draw edges
        for edge in G.edges():
            x_coords = [pos_vibrating[edge[0]][0], pos_vibrating[edge[1]][0]]
            y_coords = [pos_vibrating[edge[0]][1], pos_vibrating[edge[1]][1]]
            ax.plot(x_coords, y_coords, 'w-', linewidth=2, alpha=0.3)

        # Draw nodes
        node_colors = cm.coolwarm((eigenvector - eigenvector.min()) /
                                  (eigenvector.max() - eigenvector.min()))

        for i, (node, (x, y)) in enumerate(pos_vibrating.items()):
            ax.scatter(x, y, s=200, c=[node_colors[i]],
                      edgecolors=WHITE, linewidth=2, zorder=10)

        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        ax.set_title(f'MODE {mode_idx+1}\nλ_{mode_idx+1} = {eigenvalues[mode_idx]:.3f}',
                    fontsize=12, color=YELLOW, weight='bold')

    fig.suptitle('LAPLACIAN EIGENMODES: The Vibrational DNA of Structure',
                fontsize=16, color=YELLOW, weight='bold', y=0.98)

anim4 = FuncAnimation(fig, animate_eigenmodes, frames=50, interval=200)
print("   Rendering...")
anim4.save('/content/EIGENMODE_VISUALIZATION.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ Saved\n")

# ============================================================================
# ANIMATION 5: SPECTRAL_ATTENTION
# ============================================================================

print("🎬 Animation 5/8: SPECTRAL_ATTENTION.gif")
print("   Concept: Attention pattern based on spectral distance")
print("   Duration: 12 seconds")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), facecolor='#0E1117')
for ax in [ax1, ax2]:
    ax.set_facecolor('#0E1117')

def animate_spectral_attention(frame):
    ax1.clear()
    ax2.clear()
    for ax in [ax1, ax2]:
        ax.set_facecolor('#0E1117')

    # Focus node changes over time
    focus_node = int((frame / 60) * n_nodes) % n_nodes

    # Standard attention (content-based, random)
    standard_attention = np.random.rand(n_nodes)
    standard_attention[focus_node] = 1.0
    standard_attention = standard_attention / standard_attention.sum()

    # Spectral attention (distance-based)
    focus_pos = eigenvectors[focus_node, :3]
    spectral_distances = np.linalg.norm(eigenvectors[:, :3] - focus_pos, axis=1)
    spectral_attention = np.exp(-spectral_distances * 2)
    spectral_attention = spectral_attention / spectral_attention.sum()

    # Plot 1: Standard attention
    ax1.axis('off')
    for edge in G.edges():
        x_coords = [pos[edge[0]][0], pos[edge[1]][0]]
        y_coords = [pos[edge[0]][1], pos[edge[1]][1]]
        ax1.plot(x_coords, y_coords, 'w-', linewidth=1, alpha=0.2)

    for i, (node, (x, y)) in enumerate(pos.items()):
        size = 100 + standard_attention[i] * 500
        alpha = 0.3 + standard_attention[i] * 0.7
        color = CYAN if i == focus_node else WHITE
        ax1.scatter(x, y, s=size, c=color, alpha=alpha,
                   edgecolors=WHITE, linewidth=2)

    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_title('STANDARD ATTENTION\nContent-based (random)',
                 fontsize=14, color=CYAN, weight='bold')

    # Plot 2: Spectral attention
    ax2.axis('off')
    for edge in G.edges():
        x_coords = [pos[edge[0]][0], pos[edge[1]][0]]
        y_coords = [pos[edge[0]][1], pos[edge[1]][1]]
        ax2.plot(x_coords, y_coords, 'w-', linewidth=1, alpha=0.2)

    for i, (node, (x, y)) in enumerate(pos.items()):
        size = 100 + spectral_attention[i] * 500
        alpha = 0.3 + spectral_attention[i] * 0.7
        color = MAGENTA if i == focus_node else WHITE
        ax2.scatter(x, y, s=size, c=color, alpha=alpha,
                   edgecolors=WHITE, linewidth=2)

    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    ax2.set_title('SPECTRAL ATTENTION\nStructure-aware (geometric)',
                 fontsize=14, color=MAGENTA, weight='bold')

    fig.suptitle(f'ATTENTION FROM NODE {focus_node}\nSpectral attention respects graph geometry',
                fontsize=16, color=YELLOW, weight='bold', y=0.98)

anim5 = FuncAnimation(fig, animate_spectral_attention, frames=60, interval=200)
print("   Rendering...")
anim5.save('/content/SPECTRAL_ATTENTION.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ Saved\n")

# ============================================================================
# ANIMATIONS 6-8: Simplified for speed
# ============================================================================

print("🎬 Animations 6-8: Generating remaining animations...")

# Animation 6: Isospectral drums (simplified comparison)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7), facecolor='#0E1117')

def animate_isospectral(frame):
    for ax in [ax1, ax2]:
        ax.clear()
        ax.set_facecolor('#0E1117')
        ax.axis('off')

    # Two different shapes (simplified representation)
    # Shape 1: L-shape
    ax1.add_patch(plt.Rectangle((0, 0), 2, 1, facecolor=CYAN, alpha=0.5))
    ax1.add_patch(plt.Rectangle((0, 0), 1, 2, facecolor=CYAN, alpha=0.5))
    ax1.set_xlim(-0.5, 2.5)
    ax1.set_ylim(-0.5, 2.5)
    ax1.set_title('SHAPE A\n"L" configuration', fontsize=14, color=CYAN, weight='bold')

    # Shape 2: Different L-shape
    ax2.add_patch(plt.Rectangle((0, 0), 1, 2, facecolor=MAGENTA, alpha=0.5))
    ax2.add_patch(plt.Rectangle((0, 1), 2, 1, facecolor=MAGENTA, alpha=0.5))
    ax2.set_xlim(-0.5, 2.5)
    ax2.set_ylim(-0.5, 2.5)
    ax2.set_title('SHAPE B\n"┘" configuration', fontsize=14, color=MAGENTA, weight='bold')

    fig.suptitle('ISOSPECTRAL DRUMS\nDifferent shapes, same eigenvalues',
                fontsize=16, color=YELLOW, weight='bold')

anim6 = FuncAnimation(fig, animate_isospectral, frames=30, interval=200)
anim6.save('/content/ISOSPECTRAL_DRUMS.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ ISOSPECTRAL_DRUMS.gif saved")

# Animation 7: Drum modes (4 modes comparison)
fig, axes = plt.subplots(2, 2, figsize=(12, 12), facecolor='#0E1117')
X_drum, Y_drum, R_drum, _ = create_circular_drum_mesh(20, 40)

def animate_drum_modes(frame):
    t = frame / 30 * np.pi

    modes = [(0, 2.4048), (1, 3.8317), (0, 5.5201), (2, 5.1356)]
    titles = ['Mode (0,0)', 'Mode (1,0)', 'Mode (0,1)', 'Mode (2,0)']

    for idx, (ax, (m, freq), title) in enumerate(zip(axes.flat, modes, titles)):
        ax.clear()
        ax.set_facecolor('#0E1117')

        # Approximate mode shapes
        if m == 0:
            Z = np.cos(freq * R_drum) * np.cos(t) * 0.3
        else:
            Z = (R_drum ** m) * np.cos(freq * R_drum) * np.cos(t) * 0.3

        ax.contourf(X_drum, Y_drum, Z, levels=20, cmap='coolwarm')
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title(title, fontsize=12, color=YELLOW, weight='bold')

anim7 = FuncAnimation(fig, animate_drum_modes, frames=30, interval=200)
anim7.save('/content/DRUM_MODES.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ DRUM_MODES.gif saved")

# Animation 8: Interactive demo (eigenvalue spectrum)
fig, ax = plt.subplots(figsize=(10, 6), facecolor='#0E1117')

def animate_spectrum(frame):
    ax.clear()
    ax.set_facecolor('#0E1117')

    k_show = min(frame // 3 + 1, 10)

    ax.bar(range(1, k_show+1), eigenvalues[:k_show],
          color=CYAN, alpha=0.7, edgecolor=WHITE, linewidth=2)

    ax.set_xlabel('Eigenmode k', fontsize=12, color=WHITE, weight='bold')
    ax.set_ylabel('Eigenvalue λₖ', fontsize=12, color=WHITE, weight='bold')
    ax.set_title(f'LAPLACIAN SPECTRUM\nShowing first {k_show} eigenvalues',
                fontsize=14, color=YELLOW, weight='bold')
    ax.grid(True, alpha=0.2, color=WHITE)
    ax.set_xlim(0, 11)

anim8 = FuncAnimation(fig, animate_spectrum, frames=30, interval=200)
anim8.save('/content/INTERACTIVE_DEMO.gif', writer=PillowWriter(fps=5))
plt.close()
print("   ✓ INTERACTIVE_DEMO.gif saved")

# ============================================================================
# COMPLETION
# ============================================================================

print("\n" + "=" * 80)
print("✨ ALL ANIMATIONS GENERATED!")
print("=" * 80)
print("\nGenerated files:")
print("  1. /content/TRANSFORMER_BLINDNESS.gif")
print("  2. /content/KAC_DRUM.gif")
print("  3. /content/GRAPH_VIBRATION.gif")
print("  4. /content/EIGENMODE_VISUALIZATION.gif")
print("  5. /content/SPECTRAL_ATTENTION.gif")
print("  6. /content/ISOSPECTRAL_DRUMS.gif")
print("  7. /content/DRUM_MODES.gif")
print("  8. /content/INTERACTIVE_DEMO.gif")
print("\n📥 Download from Files panel: /content/")
print("\n🎨 Ready to embed in Medium article!")
print("=" * 80)

🎵 LAPLACIAN POSITIONAL ENCODING: ANIMATION GENERATOR

Generating 8 pedagogical animations...
Estimated time: ~30-35 minutes

🎬 Animation 1/8: TRANSFORMER_BLINDNESS.gif
   Concept: Transformer seeing graph as flat sequence
   Duration: 12 seconds
   Rendering...
   ✓ Saved

🎬 Animation 2/8: KAC_DRUM.gif
   Concept: Circular drum vibrating (fundamental mode)
   Duration: 10 seconds
   Rendering...
   ✓ Saved

🎬 Animation 3/8: GRAPH_VIBRATION.gif
   Concept: Graph 'plucked' showing vibration
   Duration: 14 seconds
   Rendering...
   ✓ Saved

🎬 Animation 4/8: EIGENMODE_VISUALIZATION.gif
   Concept: Side-by-side comparison of first 3 eigenmodes
   Duration: 10 seconds
   Rendering...
   ✓ Saved

🎬 Animation 5/8: SPECTRAL_ATTENTION.gif
   Concept: Attention pattern based on spectral distance
   Duration: 12 seconds
   Rendering...
   ✓ Saved

🎬 Animations 6-8: Generating remaining animations...
   ✓ ISOSPECTRAL_DRUMS.gif saved
   ✓ DRUM_MODES.gif saved
   ✓ INTERACTIVE_DEMO.gif saved

✨ ALL